In [50]:
import pandas as pd 

from sklearn.datasets import load_iris as load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from matplotlib.pyplot import plot, show

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [51]:
# train = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
# test = pd.read_csv('./house-prices-advanced-regression-techniques/test.csv')
from google.colab import files
uploaded = files.upload()
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [52]:
class feature_eng(dict):
    def __init__(self, data):
        self.data = data
        
    def show_columns_with_null(self):
        lst , counter = [] , 1
        for i in self.data.columns:
            if data[i].isna().sum():
                lst.append(i)
                print(f'{counter}. {i} - {self.data[i].dtype} : {self.data[i].isna().sum()} nulls')
                print('----------------------------------------')
                counter += 1
        print(f'total columns with null: {len(lst)}')
        return lst
    
    def show_columns_are_ojebct(self):
        lst , counter = [] , 1
        for i in self.data.columns:
            if self.data[i].dtype == 'object':
                lst.append(i)
                print(f'{counter}.  {i} - {self.data[i].dtype} ')
                print(f"     number of nulls: {self.data[i].isna().sum()} , number of elements: {len(self.data[i].value_counts())}")
                print('----------------------------------------')
                counter += 1
        print(f'tatol object columns : {len(lst)}')
        return lst
    
    def fill_null(self,field):
        if self.data[field].dtype == 'object':
            self.data[field].fillna('*',inplace=True)
        elif input(f"field type is {self.data[field].dtype}, do u want to fill with median?") == "yes":
            self.data[field].fillna(self.data[field].median(), inplace=True) 
            print(f'median of {field} column is {self.data[field].median()}')
        else:
            return self.data
            
    # def make_one_hot(self, field):
    #     temp = pd.get_dummies(self.data[field], prefix=field)
    #     self.data.drop(field, axis=1, inplace=True, errors='ignore')   
    #     return pd.concat([self.data, temp], axis=1)
    
    def labelencode(self,field):
        if self.data[field].dtype != 'object':
            print(f'type of this field is {self.data[field].dtype}')
            return self.data
        print(self.data[field].value_counts())
        print(f'number of titles : {len(self.data[field].value_counts())}')
        self.data[field] = LabelEncoder().fit_transform(data.Street) if input('do you want to proceed?') == 'yes' else self.data[field] 
        return self.data
    
    def labelencode_ordering(self,field):
        pass
    
    def impute(self):
        self.data = IterativeImputer().fit_transform(self.data)
        return self.data
        

In [53]:
# train.describe()
# train.info()
# test.info()
print(train.shape, test.shape)

(1460, 81) (1459, 80)


In [54]:
y_train = train['SalePrice']
id_test = test['Id']
x_train = train.drop(['SalePrice'], axis=1, errors='ignore', inplace=False)
data_row = pd.concat([x_train, test])
data = data_row.drop('Id', axis=1, errors='ignore', inplace=False)
data.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [55]:
ins = feature_eng(data)
object_list = ins.show_columns_are_ojebct()

1.  MSZoning - object 
     number of nulls: 4 , number of elements: 5
----------------------------------------
2.  Street - object 
     number of nulls: 0 , number of elements: 2
----------------------------------------
3.  Alley - object 
     number of nulls: 2721 , number of elements: 2
----------------------------------------
4.  LotShape - object 
     number of nulls: 0 , number of elements: 4
----------------------------------------
5.  LandContour - object 
     number of nulls: 0 , number of elements: 4
----------------------------------------
6.  Utilities - object 
     number of nulls: 2 , number of elements: 2
----------------------------------------
7.  LotConfig - object 
     number of nulls: 0 , number of elements: 5
----------------------------------------
8.  LandSlope - object 
     number of nulls: 0 , number of elements: 3
----------------------------------------
9.  Neighborhood - object 
     number of nulls: 0 , number of elements: 25
------------------------

In [56]:
for i in object_list:
    ins.fill_null(i)
ins.data.head(10)
ins.data.shape

(2919, 79)

In [57]:
def make_one_hot(data, field):
    temp = pd.get_dummies(data[field], prefix=field)
    data.drop(field, axis=1, inplace=True)
    return pd.concat([data, temp], axis=1)

In [58]:
for i in object_list:
  data = make_one_hot(data, i)

In [59]:
data.shape

(2919, 311)

In [60]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

data = IterativeImputer().fit_transform(data)

In [61]:
# scale(data)
x_train, x_test = data[:len(x_train)], data[len(x_train):]
x_train_scaled = scale(x_train)
x_train.shape

(1460, 311)

In [62]:
model = Sequential()
rows, cols = data.shape
model.add(Dense(1, input_dim=cols))
model.compile(loss='mse', optimizer=SGD(lr=0.001))

In [63]:
x_train_i, x_test_i , y_train_i, y_test_i = train_test_split(x_train_scaled, y_train, test_size=0.2, shuffle=True)
x_val, x_ts, y_val, y_ts = train_test_split(x_test_i, y_test_i, test_size=0.5, shuffle=True) 
print(y_val.shape, x_ts.shape, x_train_i.shape)
history_2 = model.fit(x_train_i, y_train_i, epochs=50)
print(mean_absolute_error(model.predict(x_val), y_val))
print(mean_absolute_error(model.predict(x_ts), y_ts))

(146,) (146, 311) (1168, 311)
Epoch 1/50
37/37 [==============================] - 0s 959us/step - loss: 35137380621.4737
Epoch 2/50
37/37 [==============================] - 0s 882us/step - loss: 29157798480.8421
Epoch 3/50
37/37 [==============================] - 0s 859us/step - loss: 24576010725.0526
Epoch 4/50
37/37 [==============================] - 0s 950us/step - loss: 21032112559.1579
Epoch 5/50
37/37 [==============================] - 0s 931us/step - loss: 18625171078.7368
Epoch 6/50
37/37 [==============================] - 0s 935us/step - loss: 15289836436.2105
Epoch 7/50
37/37 [==============================] - 0s 944us/step - loss: 13486470359.5789
Epoch 8/50
37/37 [==============================] - 0s 910us/step - loss: 11547157045.8947
Epoch 9/50
37/37 [==============================] - 0s 1ms/step - loss: 10085454659.3684
Epoch 10/50
37/37 [==============================] - 1s 17ms/step - loss: 8627805722.9474
Epoch 11/50
37/37 [==============================] - 0s 2ms/ste

In [64]:
history = model.fit(x_train_scaled,y_train, epochs=50)
predictions = model.predict(x_test)
new = predictions.reshape(-1)
print(new.shape)
df = pd.DataFrame(data=list(zip(id_test, new)), columns=['Id', 'SalePrice'])

# from google.colab import files
# df.to_csv('submis.csv', index=False) 
# files.download('submis.csv')
# df.head(2)

Epoch 1/50
46/46 [==============================] - 0s 917us/step - loss: 662057024.0000
Epoch 2/50
46/46 [==============================] - 0s 962us/step - loss: 619441920.0000
Epoch 3/50
46/46 [==============================] - 0s 948us/step - loss: 589582912.0000
Epoch 4/50
46/46 [==============================] - 0s 972us/step - loss: 566949760.0000
Epoch 5/50
46/46 [==============================] - 0s 913us/step - loss: 548714688.0000
Epoch 6/50
46/46 [==============================] - 0s 934us/step - loss: 535424032.0000
Epoch 7/50
46/46 [==============================] - 0s 957us/step - loss: 521601920.0000
Epoch 8/50
46/46 [==============================] - 0s 3ms/step - loss: 515206240.0000
Epoch 9/50
46/46 [==============================] - 0s 906us/step - loss: 508440960.0000
Epoch 10/50
46/46 [==============================] - 0s 962us/step - loss: 502154784.0000
Epoch 11/50
46/46 [==============================] - 0s 911us/step - loss: 495877888.0000
Epoch 12/50
46/46 [==

In [65]:
from keras import regularizers
three_layer_model = Sequential()
three_layer_model.add(Dense(100, activation='relu',input_dim=cols))
three_layer_model.add(Dense(50,activation='relu'))
three_layer_model.add(Dense(units=1, activation='linear'))
three_layer_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [66]:
history=three_layer_model.fit(x_train_i, y_train_i,epochs=50)
predictions = three_layer_model.predict(x_val)
print(mean_absolute_error(predictions,y_val))
predictions = three_layer_model.predict(x_ts)
print(mean_absolute_error(predictions,y_ts))

Epoch 1/50
37/37 [==============================] - 0s 2ms/step - loss: 40387148314.9474 - accuracy: 0.0000e+00
Epoch 2/50
37/37 [==============================] - 0s 1ms/step - loss: 38273108560.8421 - accuracy: 0.0000e+00
Epoch 3/50
37/37 [==============================] - 0s 1ms/step - loss: 40486391376.8421 - accuracy: 0.0000e+00
Epoch 4/50
37/37 [==============================] - 0s 1ms/step - loss: 39567319902.3158 - accuracy: 0.0000e+00
Epoch 5/50
37/37 [==============================] - 0s 2ms/step - loss: 39025589517.4737 - accuracy: 0.0000e+00
Epoch 6/50
37/37 [==============================] - 0s 2ms/step - loss: 38553802967.5789 - accuracy: 0.0000e+00
Epoch 7/50
37/37 [==============================] - 0s 2ms/step - loss: 37666838096.8421 - accuracy: 0.0000e+00
Epoch 8/50
37/37 [==============================] - 0s 2ms/step - loss: 37599830662.7368 - accuracy: 0.0000e+00
Epoch 9/50
37/37 [==============================] - 0s 2ms/step - loss: 37327198531.3684 - accuracy: 0.0

In [67]:
five_layer_model = Sequential()
five_layer_model.add(Dense(300, activation='relu',input_dim=cols))
five_layer_model.add(Dense(100,activation='relu'))
five_layer_model.add(Dense(50,activation='relu'))
five_layer_model.add(Dense(10,activation='relu'))
five_layer_model.add(Dense(units=1,activation='linear'))
five_layer_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [68]:
history=five_layer_model.fit(x_train_i,y_train_i,epochs=50)
predictions = five_layer_model.predict(x_val)
print(mean_absolute_error(predictions,y_val))
predictions = five_layer_model.predict(x_ts)
print(mean_absolute_error(predictions,y_ts))

Epoch 1/50
37/37 [==============================] - 1s 2ms/step - loss: 38385710026.1053 - accuracy: 0.0000e+00
Epoch 2/50
37/37 [==============================] - 0s 3ms/step - loss: 39563549642.1053 - accuracy: 0.0000e+00
Epoch 3/50
37/37 [==============================] - 0s 3ms/step - loss: 37221195075.3684 - accuracy: 0.0000e+00
Epoch 4/50
37/37 [==============================] - 0s 3ms/step - loss: 25400586994.5263 - accuracy: 0.0000e+00
Epoch 5/50
37/37 [==============================] - 0s 3ms/step - loss: 7481552033.6842 - accuracy: 0.0000e+00
Epoch 6/50
37/37 [==============================] - 0s 3ms/step - loss: 4039397234.5263 - accuracy: 0.0000e+00
Epoch 7/50
37/37 [==============================] - 0s 5ms/step - loss: 2267399417.2632 - accuracy: 0.0000e+00
Epoch 8/50
37/37 [==============================] - 0s 3ms/step - loss: 1663854908.6316 - accuracy: 0.0000e+00
Epoch 9/50
37/37 [==============================] - 0s 3ms/step - loss: 1620113556.2105 - accuracy: 0.0000e+

In [69]:
history = five_layer_model.fit(x_train_scaled,y_train, epochs=50)
predictions = five_layer_model.predict(x_test)
new = predictions.reshape(-1)
print(new.shape)
df = pd.DataFrame(data=list(zip(id_test, new)), columns=['Id', 'SalePrice'])

from google.colab import files
df.to_csv('submis.csv', index=False) 
files.download('submis.csv')
df.head(2)

Epoch 1/50
46/46 [==============================] - 0s 3ms/step - loss: 569658496.0000 - accuracy: 0.0000e+00
Epoch 2/50
46/46 [==============================] - 0s 3ms/step - loss: 504126560.0000 - accuracy: 0.0000e+00
Epoch 3/50
46/46 [==============================] - 0s 3ms/step - loss: 453873344.0000 - accuracy: 0.0000e+00
Epoch 4/50
46/46 [==============================] - 0s 3ms/step - loss: 420079168.0000 - accuracy: 0.0000e+00
Epoch 5/50
46/46 [==============================] - 0s 3ms/step - loss: 389433440.0000 - accuracy: 0.0000e+00
Epoch 6/50
46/46 [==============================] - 0s 4ms/step - loss: 378808672.0000 - accuracy: 0.0000e+00
Epoch 7/50
46/46 [==============================] - 0s 3ms/step - loss: 353487328.0000 - accuracy: 0.0000e+00
Epoch 8/50
46/46 [==============================] - 0s 3ms/step - loss: 337032000.0000 - accuracy: 0.0000e+00
Epoch 9/50
46/46 [==============================] - 0s 3ms/step - loss: 319417408.0000 - accuracy: 0.0000e+00
Epoch 10/5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Id,SalePrice
0,1461,200600496.0
1,1462,278257376.0
